In [1]:
# with open('data/all.txt','r') as datafile:
#     quotes = datafile.readlines()
# list(sorted(set(quotes[0].split())))[:145]
# words = list(sorted(set(quotes[0].split())))[138:]
# words[0] = '0' 
# words[1] = 'a'
# word_to_int = dict([(w,i) for i,w in enumerate(words)])
# int_to_word = dict([(i,w) for i,w in enumerate(words)])
# word_to_int, int_to_word
# len(int_to_word)

In [24]:
import numpy as np
import sys
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.layers import Embedding
from keras.optimizers import Adam
from keras.callbacks import LambdaCallback
from sklearn.model_selection import train_test_split

In [3]:
with open('data/all.txt','r') as quotefile:
    quotes = quotefile.readlines()

In [ ]:
with open('data/death.txt','r') as deathfile:
    deathquotes = deathfile.readlines()

In [4]:
t = Tokenizer()
t.fit_on_texts(quotes)
vocab_size = len(t.word_index) + 1

In [ ]:
# vocab_size, 
# t.word_index, 

In [6]:
index_word = dict([(i,w) for (w,i) in t.word_index.items()])

In [ ]:
np.save('index_word.npy',index_word)

In [ ]:
# integer encode the documents
encoded_docs = t.texts_to_sequences(deathquotes)
print(encoded_docs[0][:15])

In [45]:
embeddings_index = np.load('data/embeddings_index.npy')

In [46]:
embeddings_index = embeddings_index.item()

In [54]:
embeddings_index.get('the'), t.word_index['the']

(array([-0.038194  , -0.24487001,  0.72812003, -0.39961001,  0.083172  ,
         0.043953  , -0.39140999,  0.3344    , -0.57545   ,  0.087459  ,
         0.28786999, -0.06731   ,  0.30906001, -0.26383999, -0.13231   ,
        -0.20757   ,  0.33395001, -0.33848   , -0.31742999, -0.48335999,
         0.1464    , -0.37303999,  0.34577   ,  0.052041  ,  0.44946   ,
        -0.46970999,  0.02628   , -0.54154998, -0.15518001, -0.14106999,
        -0.039722  ,  0.28277001,  0.14393   ,  0.23464   , -0.31020999,
         0.086173  ,  0.20397   ,  0.52623999,  0.17163999, -0.082378  ,
        -0.71787   , -0.41531   ,  0.20334999, -0.12763   ,  0.41367   ,
         0.55186999,  0.57907999, -0.33476999, -0.36559001, -0.54856998,
        -0.062892  ,  0.26583999,  0.30204999,  0.99774998, -0.80480999,
        -3.0243001 ,  0.01254   , -0.36941999,  2.21670008,  0.72201002,
        -0.24978   ,  0.92136002,  0.034514  ,  0.46744999,  1.10790002,
        -0.19358   , -0.074575  ,  0.23353   , -0.0

In [47]:
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [56]:
embedding_matrix[9818]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [49]:
# np.save('word_index.npy',t.word_index)
np.save('embedding_matrix.npy',embedding_matrix)

In [50]:
embedding_matrix = np.load('embedding_matrix.npy')
embedding_matrix.shape

(13662, 100)

## Generate Sequences, Next Sequences and Classes

## Death Doc

In [ ]:
maxlen = 100
step = 1
seq_death = []
next_seq_death = []

In [ ]:
death_doc = encoded_docs[0]
quote_len_death = len(death_doc)

In [ ]:
for i in range(0, quote_len_death - maxlen, step):
    seq_death.append(death_doc[i: i + maxlen])
    next_seq_death.append(death_doc[i + maxlen])

print('nb sequences:', len(seq_death))

seq_death = np.asarray(seq_death)
next_seq_death = np.asarray(next_seq_death)

In [ ]:
labels_death = np.zeros(next_seq_death.shape)

In [ ]:
labels_death

## Funny Doc

In [13]:
with open('data/funny.txt','r') as funnyfile:
    funnyquotes = funnyfile.readlines()

In [14]:
encoded_docs = t.texts_to_sequences(funnyquotes)
funny_doc = encoded_docs[0]

In [15]:
maxlen = 100
step = 1
seq_funny = []
next_seq_funny = []

quote_len_funny = len(funny_doc)

In [16]:
for i in range(0, quote_len_funny - maxlen, step):
    seq_funny.append(funny_doc[i: i + maxlen])
    next_seq_funny.append(funny_doc[i + maxlen])

print('nb sequences:', len(seq_funny))

seq_funny = np.asarray(seq_funny)
next_seq_funny = np.asarray(next_seq_funny)

nb sequences: 24396


In [17]:
labels_funny = np.ones(next_seq_funny.shape)

In [18]:
labels_funny

array([ 1.,  1.,  1., ...,  1.,  1.,  1.])

## Combine

In [ ]:
X = np.vstack([seq_death,seq_funny])

In [ ]:
X.shape

In [ ]:
labels_death = np.reshape(labels_death,(labels_death.shape[0],1))
labels_funny  = np.reshape(labels_funny,(labels_funny.shape[0],1))

In [ ]:
y = np.vstack([labels_death,labels_funny])

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

# Sequence Classification Keras

In [ ]:
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False)

In [ ]:
model = Sequential()
model.add(e)

In [ ]:
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))

In [ ]:
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=24)
'''Train on 38108 samples, validate on 9528 samples
Epoch 1/10
38108/38108 [==============================] - 80s 2ms/step - 
loss: 0.0391 - acc: 0.9859 - val_loss: 4.4262e-04 - val_acc: 0.9999'''

# Text Generation using Word Embeddings

In [19]:
seq_funny, next_seq_funny

(array([[    6,   343,    77, ...,    57,  1419,    40],
        [  343,    77,     5, ...,  1419,    40,    42],
        [   77,     5,    41, ...,    40,    42,  1038],
        ..., 
        [   73,    13, 10360, ...,  3394,  1798,    23],
        [   13, 10360,     8, ...,  1798,    23,     1],
        [10360,     8,     1, ...,    23,     1,    53]]),
 array([  42, 1038,   75, ...,    1,   53, 1239]))

In [20]:
from keras.utils import to_categorical

In [21]:
y = to_categorical(next_seq_funny, num_classes=vocab_size)

In [22]:
X = seq_funny

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [38]:
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=maxlen, trainable=False)

In [39]:
model = Sequential()
model.add(e)

In [40]:
# model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
# model.add(MaxPooling1D(pool_size=2))

In [41]:
#model.add(LSTM(100, return_sequences=True)) #
model.add(LSTM(100))
model.add(Dense(y.shape[1]))

In [42]:
model.add(Activation('softmax'))
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

model.compile(
    loss='categorical_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          1366200   
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 13662)             1379862   
_________________________________________________________________
activation_2 (Activation)    (None, 13662)             0         
Total params: 2,826,462
Trainable params: 1,460,262
Non-trainable params: 1,366,200
_________________________________________________________________
None


In [43]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [44]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          epochs=10, 
          batch_size=24, #64
          callbacks=[print_callback])

Train on 19516 samples, validate on 4880 samples
Epoch 1/10
19516/19516 [==============================] - 130s 7ms/step - loss: 6.7342 - acc: 0.0378 - val_loss: 6.4484 - val_acc: 0.0402

----- Generating text after Epoch: 0
----- diversity: 0.2
[52, 108, 1641, 37, 1341, 37, 10038, 475, 8, 11, 10, 154, 776, 9, 2493, 11, 52, 2, 13, 115, 4923, 64, 10, 348, 11, 37, 9, 99, 475, 228, 102, 7, 360, 7, 139, 13, 22, 2949, 22, 3149, 10039, 7, 59, 245, 863, 4010, 7, 17, 96, 3149, 35, 15, 286, 1679, 280, 4, 7, 38, 250, 10, 9, 598, 5, 1336, 4, 53, 23, 6, 114, 6535, 227, 3, 601, 4, 61, 1, 83, 11, 5, 53, 93, 10, 38, 178, 61, 100, 4, 126, 12, 890, 23, 15, 4686, 2701, 40, 11, 133, 824, 7, 290]
----- Generating with seed: ['has', 'some', 'dramatic', 'or', 'violent', 'or', 'sinister', 'stuff', 'in', 'it', 'you', 'cant', 'forget', 'that', 'primarily', 'it', 'has', 'to', 'be', 'even', 'funnier', 'than', 'you', 'read', 'it', 'or', 'that', 'other', 'stuff', 'doesnt', 'work', 'i', 'wish', 'i', 'could', 'be', 

KeyboardInterrupt: 

In [32]:
index_word = np.load('index_word.npy')
index_word = index_word.item()

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    # preds = preds[1:]
    # preds = np.asarray(preds).astype('float64')
    # preds = np.log(preds) / temperature
    # exp_preds = np.exp(preds)
    # preds = exp_preds / np.sum(exp_preds)
    # probas = np.random.multinomial(1, preds, 1)
    return np.argmax(preds)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = np.random.randint(0, len(funny_doc) - maxlen - 1)
    for diversity in [0.2]: # 0.5, 1.0, 1.2
        print('----- diversity:', diversity)

        generated = ''
        sentence = funny_doc[start_index: start_index + maxlen]
        print(sentence)
        generated.join([str([index_word[value]]).join(' ') for value in sentence])
        print('----- Generating with seed: %s'%[index_word[word] for word in sentence])
        #sys.stdout.write(generated)

        for i in range(20):
            x_pred = np.reshape(sentence,(1, maxlen))

            preds = model.predict(x_pred, verbose=0)
            preds = preds[0]
            # print(preds.shape)
            next_index = sample(preds, diversity)
            #print(next_index)
            next_char = index_word[next_index]

            generated.join(str(next_char))
            sentence = np.append(sentence[1:],next_index)

            sys.stdout.write(next_char)
            sys.stdout.write(" ")
            sys.stdout.flush()
        print()